## Next Word Prediction 
### This exercise aims at feature engineering, understanding basic LSTM models and cheking the prediction values

In [1]:
path='1661-0.txt'
text=open(path,encoding='utf-8').read().lower()
print("corpous length",len(text)) #Entire document is called Corpus D

corpous length 581888


In [2]:
from nltk.tokenize import RegexpTokenizer

#Split the document into tokens and remove any special charcaters if any.
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)
words

['project',
 'gutenberg',
 's',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'by',
 'arthur',
 'conan',
 'doyle',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 'you',
 'may',
 'copy',
 'it',
 'give',
 'it',
 'away',
 'or',
 're',
 'use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www',
 'gutenberg',
 'net',
 'title',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'author',
 'arthur',
 'conan',
 'doyle',
 'release',
 'date',
 'november',
 '29',
 '2002',
 'ebook',
 '1661',
 'last',
 'updated',
 'may',
 '20',
 '2019',
 'language',
 'english',
 'character',
 'set',
 'encoding',
 'utf',
 '8',
 'start',
 'of',
 'this',
 'project',
 'gutenberg',
 'ebook',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'produced',
 'by',
 'an',
 'ano

In [3]:
import numpy as np
#Feature Engineering of words: Based on the problem statement we build features to train the ML model.
#1. Form the unique words as dictionar
unique_words=np.unique(words)
unique_word_index=dict((c,i) for i,c in enumerate(unique_words)) #(word: index(0 to length of dataset))
unique_word_index
#2. Create 2 features next_word and prev_word , where prev_words will keep 5 previous words and next_words in the corresponding array.
WORD_LENGTH=5
prev_words=[]
next_words=[]
for i in range(len(words)-WORD_LENGTH):
#     print(words)
#     print(words[1:1+5]) #words[start_index:end_index-1]
#     print(words[1+5])   #words[end_index]
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i+WORD_LENGTH])
print(prev_words[0]) #will have 5 words
print(next_words[0]) #will have the 6th word

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [4]:
print(len(prev_words)) #No of rows untill the end of document
print(len(unique_words)) #No of unique words
print(len(next_words))
#X direction-> All the words 
#y direction-> Each unique word
#Z direction-> Bag of words reprenstation for each word.

109221
8201
109221


In [7]:
#Bag of words for each word where the word is present keep 1.
X=np.zeros((len(prev_words),WORD_LENGTH,len(unique_words)),dtype=bool) #shape=(109221,5,8201) all with zeroes
Y=np.zeros((len(next_words),len(unique_words)),dtype=bool) #shape=(109221,8201)

#I will iterate x and y if the word is available so that the corresponding position becomes 1 in zth direction.
for i ,each_word in enumerate(prev_words):
    for j,each in enumerate(each_word):
        X[i,j,unique_word_index[each]]=1
    Y[i,unique_word_index[next_words[i]]]=1

In [10]:
print(X[0][0])
print(X.shape)

[False False False ... False False False]
(109221, 5, 8201)


In [13]:
#https://www.youtube.com/watch?v=S0XFd0VMFss
#https://www.youtube.com/watch?v=AsNTP8Kwu80
#RNN
#best one is LSTM for RNN
#RNN->LSTM->Transformers
#1. The model starts with Sequential and then LSTM can be middle layer . Dense layer is where the actual computation
#of sigma and tahh happend . 
#2.Activation functions
#1. relu [-5,0,9]-> [0,0,9] common one used with sense layer
#2. sigmoid [-6,-7,0,9] -> [0.0008,0.0007,1.00,1.0098] ;value[<-5]~0;value[>5]~1 used of binary classification and multi-label
#3. softmax[-8,-9,0,8]-> [0.2,0.5,0.1,0.2]=1 returns the probability of words and sums to 1.

from keras.models import Sequential,load_model
from keras.layers import LSTM
from keras.layers.core import Dense,Activation
from keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words)))) # 5,8201 
model.add(Dense(len(unique_words),activation="softmax")) #8201
model.summary() 
#LSTM layer returns (x,number of units) . Dense layer returns (x,number of unique words)
#Here the last layer was softmax because we want to predict one word from 8201 words with maximum probability.

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               4264960   
                                                                 
 dense_2 (Dense)             (None, 8201)              1057929   
                                                                 
Total params: 5,322,889
Trainable params: 5,322,889
Non-trainable params: 0
_________________________________________________________________


In [14]:
#optimizer is gradient desecnt approach. Adam is most common one which has weighted sum of prev value.
#RMSprop is simple gradient descent
optimizer = RMSprop(learning_rate=0.01)
#categorical_crossentropy Used as a loss function for multi-class classification model where there are two or more output labels
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history


Epoch 1/2


C:\Users\dreddyag\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


811/811 [==============================] - 78s 94ms/step - loss: 6.0109 - accuracy: 0.1064 - val_loss: 7.0681 - val_accuracy: 0.1078
Epoch 2/2
811/811 [==============================] - 88s 108ms/step - loss: 5.7739 - accuracy: 0.1475 - val_loss: 8.0000 - val_accuracy: 0.1084


In [ ]:
X=np.zeros((len(prev_words),WORD_LENGTH,len(unique_words)),dtype=bool) #shape=(109221,5,8201) all with zeroes
Y=np.zeros((len(next_words),len(unique_words)),dtype=bool) #shape=(109221,8201)

#I will iterate x and y if the word is available so that the corresponding position becomes 1 in zth direction.
for i ,each_word in enumerate(prev_words):
    for j,each in enumerate(each_word):
        X[i,j,unique_word_index[each]]=1

In [62]:
def predict_completions(text,word_length):
    #1.Split the text
    word=text.split()
    #2.create zero matrix (1 sentence, 5 length, vocabulary length)
    test_data=np.zeros((1,word_length,len(unique_words)),dtype=bool)
    #3.keep 1 where ever word is present
    for i,w in enumerate(word):
        test_data[0,i,unique_word_index[w]]=1
    #4.Predict the test data
    y_hat=model.predict(test_data)
    #5.it returns probabilities of words and the one with maximum probability is the word
    y_hat_index=np.argmax(y_hat)
    #6.Lookup the word is maximum prob in vocabulary
    return unique_words[y_hat_index]

In [66]:
boolx="N"
while boolx!="Y":
    try:
        seq=input("Enter Someting: ").lower()
    except:
        seq="It is not a lack of love, but a lack of"
    print(predict_completions(seq, 5))
    boolx=input("To Exit Press : (Y)")

Enter Someting: I love project gutenberg
1/1 [==============================] - 0s 15ms/step
and
To Exit Press : (Y)N
Enter Someting: project gutenberg s the project 
1/1 [==============================] - 0s 15ms/step
of
To Exit Press : (Y)n
Enter Someting: project gutenberg s the ebook
1/1 [==============================] - 0s 19ms/step
of
To Exit Press : (Y)Y
